
# **Running Pyspark in Colab**

java installation
spark and hadoop installation

above are the one time activity

# One time activity

In [1]:
!sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [2]:
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz

In [3]:
!tar xf spark-2.4.7-bin-hadoop2.7.tgz

In [4]:
!pip install -q findspark

# spark context set
 Set the location of Java and Spark  and then creating the spark context so that it will be availaible for precessing.
 Its a singleton object so same context is availaible throughout the session 

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [6]:
import findspark

In [7]:
! ls /content

sample_data  spark-2.4.7-bin-hadoop2.7	spark-2.4.7-bin-hadoop2.7.tgz


In [8]:
findspark.init()

In [9]:
from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()

In [10]:
# this activity will be done by dono team as well
spark = SparkSession.builder.master("local").appName("max_pyspark").config('spark.ui.port', '4050').getOrCreate()

In [11]:
spark

# creating a public url using ngrok service for yarn UI

In [12]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2021-01-17 14:21:10--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.200.171.63, 52.23.32.39, 52.54.16.202, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.200.171.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  73.9MB/s    in 0.2s    

2021-01-17 14:21:11 (73.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [13]:
get_ipython().system_raw('./ngrok http 4050 &')

In [14]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}



# spark analysis
groupby and other aggregation functions

In [17]:
df = spark.read.csv("./sample_data/california_housing_train.csv", header=True, inferSchema=True)
# df = spark.read.csv("./sample_data/blk_fdy_train.csv", header=True, inferSchema=True)

In [18]:
type(df)

pyspark.sql.dataframe.DataFrame

In [19]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [ ]:
df.show(5)

In [ ]:
df.describe().show()

In [ ]:
df.select("City_Category").distinct().show()

In [ ]:
df.groupby("User_ID").agg(F.sum(F.col("Purchase").alias("agg_sum"))).show()

In [ ]:
from pyspark.sql import functions as F
df.groupBy("City_Category").agg(F.sum("Purchase")).show()

In [ ]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

In [ ]:
df

In [ ]:
df = df.fillna({'Product_Category_2':0, 'Product_Category_3':0})
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

# File writing to csv in spark

In [ ]:

# df.write.csv("/content/drive/My Drive/AV articles/PySpark on Colab/preprocessed_data")
# df.rdd.getNumPartitions()

In [ ]:

# Spark df to Pandas df
df_pd = df.toPandas()

# Store result
df_pd.to_csv("/content/drive/My Drive/AV articles/PySpark on Colab/pandas_preprocessed_data.csv")

In [ ]:
!ls ./sample_data/california_housing_train.csv

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md



# Linear Regression Model




In [16]:
from google.colab import files
files.upload()

{}

Check the dataset is uploaded correctly in the system by the following command

In [ ]:
!ls

BostonHousing.csv  spark-2.3.2-bin-hadoop2.7
sample_data	   spark-2.3.2-bin-hadoop2.7.tgz


 For our linear regression model we need to import two modules from Pyspark i.e. Vector Assembler and Linear Regression. Vector Assembler is a transformer that assembles all the features into one vector from multiple columns that contain type double. We could have used StringIndexer if any of our columns contains string values to convert it into numeric values.

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

dataset = spark.read.csv('BostonHousing.csv',inferSchema=True, header =True)

In [ ]:
dataset.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



convert all the features from different columns into a single column and naming new vector column as 'Attributes' in the outputCol.

In [ ]:
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')

output = assembler.transform(dataset)

#Input vs Output
finalized_data = output.select("Attributes","medv")

finalized_data.show()

+--------------------+----+
|          Attributes|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796,0.0,8.14...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



'Attributes' are in the input features from all the columns and 'medv' is the target column.
 spliting the training and testing data according to our dataset (0.8 and 0.2)

In [ ]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])


regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

#Learn to fit the model from training set
regressor = regressor.fit(train_data)

#To predict the prices on testing set
pred = regressor.evaluate(test_data)

#Predict the model
pred.predictions.show()

+--------------------+----+------------------+
|          Attributes|medv|        prediction|
+--------------------+----+------------------+
|[0.01301,35.0,1.5...|32.7| 30.07670363535312|
|[0.01538,90.0,3.7...|44.0| 37.75244575519337|
|[0.01778,95.0,1.4...|32.9|30.596108327253294|
|[0.0187,85.0,4.15...|23.1|25.717620889129734|
|[0.01965,80.0,1.7...|20.1|19.992379582220035|
|[0.02729,0.0,7.07...|34.7|30.425294527192754|
|[0.03113,0.0,4.39...|17.5|16.330496893793097|
|[0.03237,0.0,2.18...|33.4|28.578543755284294|
|[0.03306,0.0,5.19...|20.6| 22.16010760013387|
|[0.03359,75.0,2.9...|34.9| 34.42265990782376|
|[0.03537,34.0,6.0...|22.0|28.784081950984906|
|[0.03584,80.0,3.3...|23.5| 30.77179427151925|
|[0.03738,0.0,5.19...|20.7| 21.65956978285279|
|[0.04297,52.5,5.3...|24.8|26.706348196385573|
|[0.0456,0.0,13.89...|23.3|26.369847201011538|
|[0.04684,0.0,3.41...|22.6|26.949731074397704|
|[0.04981,21.0,5.6...|23.4| 23.90871028835852|
|[0.05372,0.0,13.9...|27.1|27.156639422924407|
|[0.05425,0.0

In [ ]:
#coefficient of the regression model
coeff = regressor.coefficients

#X and Y intercept
intr = regressor.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)


The coefficient of the model is : DenseVector([-0.1239, 0.056, 0.0205, 2.7283, -16.8634, 3.218, 0.0163, -1.4331, 0.3657, -0.0134, -0.9328, 0.0096, -0.6229])
The Intercept of the model is : 39.049826


# Basic Statistical Analysis


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)



RMSE: 4.703
MSE: 22.118
MAE: 3.457
r2: 0.738
